In [1]:
#
# classifer notebook
#
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Lambda, Compose
from LandmarkDataset import LandmarkDataset

#
# MODEL
#

class NN(torch.nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)
        #self.fc2 = torch.nn.Linear(hidden_size, 48)
        #self.fc3 = torch.nn.Linear(48, num_classes)
        self.dropout = torch.nn.Dropout(p=0.05)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.fc2(out)
        #out = self.relu(out)
        #out = self.dropout(out)
        
        #out = self.fc3(out)
        
        return out

    
#
# PARAMS
#

batch_size = 4
hidden_size = 96
learning_rate = .01
num_epochs = 250


#
# INITIAL DATA
#

# for directory load each file
# generate mapping of file -> class -> idx

transformations = Compose([
    Lambda(lambda x: torch.tensor(x.values).float())
])

dataset = LandmarkDataset("/home/jovyan/train/data/",
                          transform=transformations)

num_classes = dataset.num_class
input_size = dataset.input_size #2 * (21 * 3) + 12 #138

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)



#
# INIT MODELS, LOSS FN, GRAD
#

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NN(input_size, hidden_size, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

#
# TRAIN
#

running_loss_epoch = 0
for epoch in range(num_epochs):
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  

        # zero out accumulated gradients
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(landmarks)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        # NB: len(dataloader) is num of batches
        running_loss_epoch += loss.item()
        if ((epoch+1) % 10 == 0) and ((batch_idx+1) % len(dataloader) == 0):
            print ('Epoch [{}/{}], Loss: {:.4f}, Running Loss (per N epoch): {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item(), running_loss_epoch))
            running_loss_epoch = 0


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch [10/250], Loss: 0.0470, Running Loss (per N epoch): 763.1421
Epoch [20/250], Loss: 1.3147, Running Loss (per N epoch): 372.3948
Epoch [30/250], Loss: 0.7448, Running Loss (per N epoch): 290.8667
Epoch [40/250], Loss: 0.0027, Running Loss (per N epoch): 254.9774
Epoch [50/250], Loss: 1.4918, Running Loss (per N epoch): 236.0235
Epoch [60/250], Loss: 0.0967, Running Loss (per N epoch): 223.3853
Epoch [70/250], Loss: 0.3164, Running Loss (per N epoch): 196.8412
Epoch [80/250], Loss: 0.0512, Running Loss (per N epoch): 200.2463
Epoch [90/250], Loss: 0.2299, Running Loss (per N epoch): 214.6818
Epoch [100/250], Loss: 1.0209, Running Loss (per N epoch): 175.5039
Epoch [110/250], Loss: 0.0000, Running Loss (per N epoch): 188.6002
Epoch [120/250], Loss: 0.0184, Running Loss (per N epoch): 173.6212
Epoch [130/250], Loss: 0.1012, Running Loss (per N epoch): 186.1895
Epoch [140/250], Loss: 0.0189, Running Loss (per N epoch): 185.8715
Epoch [150/250], Loss: 0.4102, Running Loss (per N epoch)

In [2]:
#
# EXPORT
#
# NB: LandmarkDataset.py updates meta.json with class index
#
dummy_input = torch.zeros(input_size)
#model.load_state_dict(torch.load('./model_overfit.pt'))
torch.onnx.export(model, dummy_input, 'onnx_model.onnx', export_params=True,
                  input_names = ['landmarks'], output_names = ['class'], verbose=True)

#import onnx
#onnx_model = onnx.load("./onnx_model.onnx")
#onnx.checker.check_model(onnx_model)


Exported graph: graph(%landmarks : Float(138, strides=[1], requires_grad=0, device=cpu),
      %fc1.bias : Float(96, strides=[1], requires_grad=1, device=cpu),
      %fc2.bias : Float(14, strides=[1], requires_grad=1, device=cpu),
      %onnx::MatMul_12 : Float(138, 96, strides=[1, 138], requires_grad=0, device=cpu),
      %onnx::MatMul_13 : Float(96, 14, strides=[1, 96], requires_grad=0, device=cpu)):
  %onnx::Add_6 : Float(96, strides=[1], device=cpu) = onnx::MatMul[onnx_name="MatMul_0"](%landmarks, %onnx::MatMul_12) # /opt/conda/lib/python3.10/site-packages/torch/nn/modules/linear.py:114:0
  %input : Float(96, strides=[1], requires_grad=1, device=cpu) = onnx::Add[onnx_name="Add_1"](%fc1.bias, %onnx::Add_6) # /opt/conda/lib/python3.10/site-packages/torch/nn/modules/linear.py:114:0
  %input.3 : Float(96, strides=[1], requires_grad=1, device=cpu) = onnx::Relu[onnx_name="Relu_2"](%input) # /opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1453:0
  %onnx::Add_10 : Float(14, 

In [3]:
#
# EVAL
#
torch.set_printoptions(precision=4, sci_mode=False)
softmax = torch.nn.Softmax(dim=1)
accuracy = 0
count = 0
with torch.no_grad():
    
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  
        out = model(landmarks)
        prob = softmax(out.data)     #setup for threshold or 'garbage' class
        _, klass = torch.max(out.data, 1)
        
        print(klass, labels, klass==labels)
        print(prob)
        print("-----")
        
        accuracy += (klass == labels).sum().item()
        count += len(labels)
        

print('--------')
print("Accuracy {}/{} : {:.4f}".format(accuracy, count, accuracy/count))


tensor([2, 2, 0, 6]) tensor([2, 2, 1, 6]) tensor([ True,  True, False,  True])
tensor([[    0.0000,     0.0000,     0.9996,     0.0000,     0.0000,     0.0000,
             0.0004,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
             0.0000,     0.0000],
        [    0.0000,     0.0000,     0.6344,     0.0079,     0.0008,     0.0000,
             0.3371,     0.0004,     0.0000,     0.0001,     0.0193,     0.0000,
             0.0000,     0.0000],
        [    0.5180,     0.3274,     0.0016,     0.0073,     0.0209,     0.0025,
             0.0281,     0.0053,     0.0021,     0.0562,     0.0059,     0.0059,
             0.0169,     0.0020],
        [    0.0000,     0.0000,     0.0810,     0.0000,     0.0055,     0.0000,
             0.9032,     0.0000,     0.0103,     0.0000,     0.0000,     0.0000,
             0.0000,     0.0000]])
-----
tensor([2, 2, 8, 4]) tensor([2, 2, 4, 4]) tensor([ True,  True, False,  True])
tensor([[    0.0000,     0.0000,     1.0000,     0.

In [4]:
print("Done")

Done
